# Concentration Jump Refinement

This notebook demonstrates the detection and refinement of concentration jumps in a phase diagram, specifically for a model with a miscibility gap.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from landau.phases import LinePhase, SlowInterpolatingPhase
from landau.calculate import calc_phase_diagram, refine_concentration_jumps

## Set up the phase model

We use a `SlowInterpolatingPhase` with regular solution behavior. We set an interaction parameter $W$ such that demixing occurs below a critical temperature $T_c$.

In [ ]:
p0 = LinePhase("A", 0, 0, 0)
p1 = LinePhase("B", 1, 0, 0)
p_mid = LinePhase("Mid", 0.5, 0.025, 0) # W = 0.1 eV

phase = SlowInterpolatingPhase(
    name="MiscibilityGap",
    phases=[p0, p_mid, p1],
    add_entropy=True
)

Tc = 0.1 / (2 * 8.617e-5) # approx 580 K

## 1D Refinement: Isothermal

We calculate the concentration along a chemical potential line at $T = 300$ K.

In [ ]:
T = 300
mus = np.linspace(-0.05, 0.05, 100)
df = calc_phase_diagram([phase], T, mus)

plt.plot(df.mu, df.c, 'o-', label=f'T={T} K')
plt.xlabel(r'$\mu$ (eV)')
plt.ylabel('Concentration c')
plt.legend()
plt.title('Concentration vs Chemical Potential')

### Refine the jump

Now we apply the `refine_concentration_jumps` function to find the exact location of the jump.

In [ ]:
cl, cr, mu_j, Tj = refine_concentration_jumps(
    df['T'].values, df.mu.values, df.c.values, phase
)

print(f"Refined jump at mu = {mu_j[0]:.6f} eV")
print(f"c_left = {cl[0]:.4f}, c_right = {cr[0]:.4f}")

plt.plot(df.mu, df.c, 'o-', label='Gridded data')
for m in mu_j:
    plt.axvline(m, color='red', linestyle='--', label='Refined jump')
plt.xlabel(r'$\mu$ (eV)')
plt.ylabel('Concentration c')
plt.legend()
plt.title('Refined Jump Location')

## 2D Refinement: Multi-temperature

We test the refinement on a range of temperatures crossing $T_c$.

In [ ]:
Ts = np.linspace(300, 700, 10)
mus = np.linspace(-0.05, 0.05, 50)
df_2d = calc_phase_diagram([phase], Ts, mus)

cl_2d, cr_2d, mu_j_2d, Tj_2d = refine_concentration_jumps(
    df_2d['T'].values, df_2d.mu.values, df_2d.c.values, phase
)

plt.figure(figsize=(10, 6))
plt.scatter(df_2d['T'], df_2d.c, c=df_2d.mu, cmap='viridis', alpha=0.3, label='Gridded points')
plt.colorbar(label='mu (eV)')
plt.scatter(Tj_2d, cl_2d, color='red', marker='x', label='Refined Jump (left)')
plt.scatter(Tj_2d, cr_2d, color='blue', marker='x', label='Refined Jump (right)')
plt.xlabel('Temperature T (K)')
plt.ylabel('Concentration c')
plt.title('2D Refinement: Concentration Jumps over Temperature')
plt.legend()
plt.show()